### import needed libraries

In [452]:
import seaborn as sns
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np


### function for counting rows with shared values

In [453]:
def count_shared_values(df, col1, col2, val1, val2):
    shared_count = len(df[(df[col1] == val1) & (df[col2] == val2)])
    return shared_count


### read in .csv file

In [454]:
df = pd.read_csv('rr_data.csv', low_memory=False)
df.head(10)

,Reporting Railroad Name,Maintenance Railroad Name,Date,Accident Type,Hazmat Released Cars,Persons Evacuated,State Name,Weather Condition,Train Speed,Gross Tonnage,...,Remote Control Locomotive Code,Positive Alcohol Tests,Positive Drug Tests,Passengers Transported,Equipment Damage Cost,Primary Accident Cause,Hours Engineers On Duty,Total Persons Killed,Class,Equipment Type Code
0,Northern Indiana Commuter Transportation District,NORTHEAST ILLINOIS REGIONAL COMMUTER RAIL CORP...,06/06/2017,Derailment,0.0,149.0,ILLINOIS,Clear,10.0,0.0,...,0.0,0.0,0.0,Yes,98627.0,Switch point worn or broken,1.0,0.0,No,3
1,NORTHEAST ILLINOIS REGIONAL COMMUTER RAIL CORP...,NORTHEAST ILLINOIS REGIONAL COMMUTER RAIL CORP...,06/06/2017,Derailment,0.0,0.0,ILLINOIS,Clear,0.0,0.0,...,NaN,NaN,NaN,NaN,0.0,Switch point worn or broken,NaN,0.0,No,NaN
2,Conrail,Conrail,04/06/1981,Side collision,0.0,0.0,NEW YORK,Snow,0.0,0.0,...,NaN,0.0,0.0,No,4000.0,Cars left foul,NaN,0.0,1L,5
3,Illinois Central Railroad Company,Cargill Elevator,01/11/2007,Derailment,0.0,0.0,LOUISIANA,Cloudy,4.0,14100.0,...,0.0,NaN,NaN,No,5220.0,Switch damaged or out of adjustment,10.0,0.0,1L,1
4,Illinois Central Railroad Company,Illinois Central Railroad Company,10/07/2017,Hwy-rail crossing,0.0,0.0,ILLINOIS,Clear,0.0,3482.0,...,0.0,NaN,NaN,No,13000.0,Highway user deliberately disregarded crossing...,8.0,0.0,Cl,1
5,Norfolk Southern Railway Company,Norfolk Southern Railway Company,10/15/2017,Derailment,0.0,0.0,ILLINOIS,Rain,3.0,6706.0,...,0.0,0.0,0.0,No,7418.0,"Shoving movement, absence of man on or at lead...",2.0,0.0,CL,1
6,Union Pacific Railroad Company,Union Pacific Railroad Company,12/04/2017,Obstruction,0.0,0.0,ILLINOIS,Cloudy,0.0,0.0,...,NaN,NaN,NaN,NaN,0.0,Object or equipment on or fouling track - othe...,NaN,0.0,CL,NaN
7,NORTHEAST ILLINOIS REGIONAL COMMUTER RAIL CORP...,NORTHEAST ILLINOIS REGIONAL COMMUTER RAIL CORP...,12/19/2016,Other (describe in narrative),0.0,0.0,ILLINOIS,Clear,0.0,0.0,...,NaN,NaN,NaN,NaN,0.0,Other miscellaneous causes (Provide detailed d...,NaN,0.0,No,NaN
8,Indiana Harbor Belt Railroad Company,Indiana Harbor Belt Railroad Company,03/30/2017,Derailment,0.0,0.0,ILLINOIS,Rain,10.0,2399.0,...,0.0,NaN,NaN,NaN,390870.0,Broken Rail - Base [Desc.Changed-Refer to Curr...,2.0,0.0,Cl,7
9,BNSF Railway Company,BNSF Railway Company,04/06/2017,Derailment,0.0,0.0,ILLINOIS,Clear,10.0,15760.0,...,0.0,0.0,0.0,No,750.0,Damaged flange or tread (build up),9.0,0.0,CL,1


### drop columns that I don't need 

In [455]:
df.drop(['Class', 'Hours Engineers On Duty', 
         'Maintenance Railroad Name', 
         'Equipment Attended', 
         'Positive Drug Tests', 
         'Positive Alcohol Tests',
         'Remote Control Locomotive Code',
         'Primary Accident Cause'
         ],  axis=1, inplace=True)


### Check DataFrame dtypes

In [456]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215849 entries, 0 to 215848
Data columns (total 13 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Reporting Railroad Name  215846 non-null  object 
 1   Date                     215847 non-null  object 
 2   Accident Type            215847 non-null  object 
 3   Hazmat Released Cars     215848 non-null  float64
 4   Persons Evacuated        215848 non-null  float64
 5   State Name               215848 non-null  object 
 6   Weather Condition        215454 non-null  object 
 7   Train Speed              215846 non-null  float64
 8   Gross Tonnage            215848 non-null  float64
 9   Passengers Transported   205176 non-null  object 
 10  Equipment Damage Cost    215848 non-null  float64
 11  Total Persons Killed     215848 non-null  float64
 12  Equipment Type Code      204640 non-null  object 
dtypes: float64(6), object(7)
memory usage: 21.4+ MB


### Rename columns

In [457]:
df.rename(columns= {'Equipment Type Code': 'Equipment', 
'Weather Condition': 'Weather', 
'Train Speed': 'Speed', 
'Reporting Railroad Name': 'Railroad', 
'State Name': 'State', 
'Equipment Damage Cost': 'Damage',
'Primary Accident Cause': 'Cause',
'Passengers Transported': 'Passengers',
'Accident Type': 'Accident_Type',
'Persons Evacuated': 'Evacuated',
'Positive Alcohol Tests': 'Alcohol_Test',
'Positive Drug Tests': 'Drug_Test',
'Hazmat Released Cars': 'Spill_Cars',
'Total Persons Killed': 'Killed',
'Gross Tonnage': 'Tonnage'}, inplace=True)

### Dropping passenger rail incidents

In [458]:
passenger_trians = [2, 3, 'B', 'C', 'EMU', 'DMU']

df = df[df.Equipment.isin(passenger_trians) == False]

In [459]:
df.drop(df.loc[df['Passengers']=='Yes'].index, inplace=True)

### Only needed Equipment and Passengers to drop all but freight

In [460]:
df.drop('Equipment', axis=1, inplace=True)
df.drop('Passengers', axis=1, inplace=True)        
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 208708 entries, 1 to 215848
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Railroad       208705 non-null  object 
 1   Date           208706 non-null  object 
 2   Accident_Type  208706 non-null  object 
 3   Spill_Cars     208707 non-null  float64
 4   Evacuated      208707 non-null  float64
 5   State          208707 non-null  object 
 6   Weather        208325 non-null  object 
 7   Speed          208705 non-null  float64
 8   Tonnage        208707 non-null  float64
 9   Damage         208707 non-null  float64
 10  Killed         208707 non-null  float64
dtypes: float64(6), object(5)
memory usage: 19.1+ MB


### Any rows with NaN for railroad name or date is a error

In [461]:
df.dropna(subset=['Railroad', 'Date'], inplace=True)

### Fill NaN with usable values

In [462]:
df.Weather = df.Weather.fillna('Unknown')
df.Accident_Type = df.Accident_Type.fillna('Unknown')
df.Speed = df.Speed.fillna(0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 208704 entries, 1 to 215848
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Railroad       208704 non-null  object 
 1   Date           208704 non-null  object 
 2   Accident_Type  208704 non-null  object 
 3   Spill_Cars     208704 non-null  float64
 4   Evacuated      208704 non-null  float64
 5   State          208704 non-null  object 
 6   Weather        208704 non-null  object 
 7   Speed          208704 non-null  float64
 8   Tonnage        208704 non-null  float64
 9   Damage         208704 non-null  float64
 10  Killed         208704 non-null  float64
dtypes: float64(6), object(5)
memory usage: 19.1+ MB


### Change data types

In [463]:
df = df.astype({'Tonnage': 'Int64', 'Damage': 'Int64', 
'Killed': 'Int64', 'Speed': 'Int64',
'Spill_Cars': 'Int64',
'Evacuated': 'Int64'})
df['Date'] = df['Date'].astype('datetime64[ns]')
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 208704 entries, 1 to 215848
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Railroad       208704 non-null  object        
 1   Date           208704 non-null  datetime64[ns]
 2   Accident_Type  208704 non-null  object        
 3   Spill_Cars     208704 non-null  Int64         
 4   Evacuated      208704 non-null  Int64         
 5   State          208704 non-null  object        
 6   Weather        208704 non-null  object        
 7   Speed          208704 non-null  Int64         
 8   Tonnage        208704 non-null  Int64         
 9   Damage         208704 non-null  Int64         
 10  Killed         208704 non-null  Int64         
dtypes: Int64(6), datetime64[ns](1), object(4)
memory usage: 20.3+ MB


### dropping the one canadian owned railroad

In [464]:
df = df[~df['Railroad'].str.contains('Illinois Central Gulf Railroad Company')]

### Dropping all but the major class 1 railroads

In [465]:
value_counts = df['Railroad'].value_counts()
to_remove = value_counts[value_counts <= 4120].index
df = df[~df.Railroad.isin(to_remove)]
df.Railroad.value_counts()

Union Pacific Railroad Company                 30701
BNSF Railway Company                           15547
CSX Transportation                             13477
Burlington Northern Railroad Company           13327
Conrail                                        12487
Southern Pacific Transportation Company         8896
Norfolk Southern Railway Company                8452
Chicago And North Western Railway Company       7743
Atchison, Topeka & Santa Fe Railway Company     5398
Louisville And Nashville Railroad Company       4141
Name: Railroad, dtype: int64

### Correcting the railroad names.

In [466]:
replace_dict = {'BNSF Railway Company': 'BNSF',
                'Burlington Northern Railroad Company': 'BNSF',
                'Conrail': 'Norfolk Southern',
                'Norfolk Southern Railway Company': 'Norfolk Southern',
                'Union Pacific Railroad Company': 'Union Pacific',
                'Southern Pacific Transportation Company': 'Union Pacific',
                'Atchison, Topeka & Santa Fe Railway Company': 'BNSF',
                'Chicago And North Western Railway Company': 'Union Pacific',
                'CSX Transportation': 'CSX',
                'Louisville And Nashville Railroad Company': 'CSX'}
df['Railroad'] = df['Railroad'].replace(replace_dict)

### Checking that the rename worked

In [467]:

value_counts = df['Railroad'].value_counts()

df.Railroad.value_counts()

Union Pacific       47340
BNSF                34272
Norfolk Southern    20939
CSX                 17618
Name: Railroad, dtype: int64

### sorting by date

In [468]:
df.sort_values(by=['Date'], inplace=True)

### reset index

In [469]:
df = df.reset_index(drop=True)
df.head(15)

,Railroad,Date,Accident_Type,Spill_Cars,Evacuated,State,Weather,Speed,Tonnage,Damage,Killed
0,BNSF,1975-01-01,Derailment,0,0,CALIFORNIA,Cloudy,0,0,1800,0
1,BNSF,1975-01-01,Derailment,0,0,ARIZONA,Cloudy,12,3826,5000,0
2,Union Pacific,1975-01-01,Derailment,0,0,ILLINOIS,Clear,5,0,1400,0
3,Union Pacific,1975-01-02,Derailment,0,0,KANSAS,Snow,45,6843,400,0
4,Union Pacific,1975-01-02,Derailment,0,0,IDAHO,Cloudy,55,6487,4800,0
5,BNSF,1975-01-03,Derailment,0,0,NEBRASKA,Cloudy,7,3995,1750,0
6,Union Pacific,1975-01-03,Derailment,0,0,TEXAS,Clear,0,0,0,0
7,BNSF,1975-01-03,Other impacts,0,0,ILLINOIS,Cloudy,50,4846,500,0
8,BNSF,1975-01-04,Side collision,0,0,WASHINGTON,Cloudy,45,10046,29850,0
9,CSX,1975-01-04,Derailment,0,0,KENTUCKY,Clear,25,3053,85583,0


## Calling my shared_count function to see that Kentucky has a total of 1928 derailments in this dataframe

In [470]:
shared_count = count_shared_values(df, 'Accident_Type', 'State', 'Derailment', 'KENTUCKY')
shared_count

1928

## CSX has had 10956 total derailments

In [471]:
shared_count = count_shared_values(df, 'Railroad', 'Accident_Type', 'CSX', 'Derailment')
shared_count

10956

## Norfolk Southern has 13617 total dedrailments

In [472]:
shared_count = count_shared_values(df, 'Railroad', 'Accident_Type', 'Norfolk Southern', 'Derailment')
shared_count

13617